# Denoising Autoencoder in PyTorch
This notebook demonstrates how to build a convolutional autoencoder to remove noise from images.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

## Define the Autoencoder

In [2]:
class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 7)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 7),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

## Load the Images into Dataset

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.ImageFolder(
    root='tiny-imagenet-200/train',
    transform=transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
)

train_loader = DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
    num_workers=2
)

# Create test dataset
test_dataset = datasets.ImageFolder(
    root='tiny-imagenet-200/val',
    transform=transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
)

test_loader = DataLoader(
    test_dataset,
    batch_size=128,
    shuffle=False,
    num_workers=2
)




# Add Noise Function

In [4]:
def add_noise(img):
    noise = torch.randn_like(img) * 0.2
    noisy_img = img + noise
    return torch.clamp(noisy_img, 0., 1.)


## Train the Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DenoisingAutoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for images, _ in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
        noisy_images = add_noise(images).to(device)
        images = images.to(device)
        
        outputs = model(noisy_images)
        loss = criterion(outputs, images)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {train_loss/len(train_loader):.4f}')

Epoch 1/10, Loss: 0.2441


Epoch 2/10, Loss: 0.2355


Epoch 3/10: 100%|█████████▉| 781/782 [00:28<00:00, 31.17it/s]

## Evaluate the Model

In [ ]:
model.eval()
with torch.no_grad():
    for images, _ in test_loader:
        noisy_images = add_noise(images).to(device)
        images = images.to(device)
        outputs = model(noisy_images)
        break

# Visualize
def show_images(images1, images2):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(np.squeeze(images1.cpu().numpy()[0]), cmap='gray')
    axes[0].set_title('Noisy Image')
    axes[1].imshow(np.squeeze(images2.cpu().numpy()[0]), cmap='gray')
    axes[1].set_title('Denoised Image')
    plt.show()

show_images(noisy_images, outputs)